Categorical Features

Transaction
ProductCD
card1
card6
addr1, addr2
P_emaildomain
R_emaildomain
M1 - M9

*   Новый пункт
*   Новый пункт
*   Новый пункт
*   Новый пункт
*   Новый пункт
*   Новый пункт
*   Новый пункт
*   Новый пункт




Categorical Features

Identity
DeviceType
DeviceInfo
id_12 - id_38

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import datetime
import gc

In [2]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from  sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

Параметры для данного ноутбука:

In [3]:
target = 'isFraud'
is_drop_collinear = True # Удалять ли коррелирующие признаки
coefficient_corr = 0.95 # Коэффициент корреляции, начиная с которого удалять признаки
nan_percent = 0.95 # Степень незаполненности данных, при которой мы относим столбец к плохим
is_drop_nan_columns = False # Удалять ли найденные признаки с плохой заполняемостью
bad_fill_category_value = '_NAN_' # Чем заполнять пропуски в категориальных признаках
bad_fill_float_value = 0 # Чем заполнять пропуски в вещественных признаках
bad_fill_numeric_value = 0 # Чем заполнять пропуски в целочисленных признаках
is_need_pca = False

Загружаем данные:

In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')
"""
df_train_transaction = pd.read_csv('gdrive/My Drive/data/gp2/train_transaction.csv')
df_train_transaction.head()"""

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


"\ndf_train_transaction = pd.read_csv('gdrive/My Drive/data/gp2/train_transaction.csv')\ndf_train_transaction.head()"

In [5]:
def get_data(transaction_csv, identity_csv,is_test = False):
  df_train_transaction = pd.read_csv(transaction_csv)
  df_train_identity = pd.read_csv(identity_csv)
  df_train = df_train_transaction.merge(df_train_identity, how = 'left',on='TransactionID')
  
  del df_train_identity,df_train_transaction
  gc.collect

  if is_test == False:
    df_y = pd.DataFrame(df_train[target])
    df_train.drop(columns = [target],axis = 1, inplace = True)
  else:
    df_y = pd.DataFrame(df_train['TransactionID'])


  df_train.drop('TransactionID', 1,inplace = True)
  df_train.drop('TransactionDT', 1,inplace = True)
  df_train.drop('DeviceType', 1,inplace = True)
  df_train.drop('DeviceInfo', 1,inplace = True)

  return df_train, df_y



In [6]:
df_train, df_y = get_data('gdrive/My Drive/data/gp2/train_transaction.csv', 
                'gdrive/My Drive/data/gp2/train_identity.csv',
                False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
"""df_train_identity = pd.read_csv('gdrive/My Drive/data/gp2/train_identity.csv')
df_train_identity.head()""""

Объединим датасеты

In [ ]:
"""print(df_train_transaction.shape,df_train_identity.shape)
df_train = df_train_transaction.merge(df_train_identity, how = 'left',on='TransactionID')
print(df_train.shape)"""

In [ ]:
#del df_train_identity,df_train_transaction
#gc.collect

Выделим целевой признак:

In [ ]:
"""df_train.drop('TransactionID', 1,inplace = True)
df_train.drop('TransactionDT', 1,inplace = True)
df_train.drop('DeviceType', 1,inplace = True)
df_train.drop('DeviceInfo', 1,inplace = True)"""

Посмотрим на корреляцию признаков

In [7]:
def remove_collinear_features(x, threshold=0.6):

    
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    
                   
    return x

In [8]:
if is_drop_collinear == True: 
  shape = df_train.shape[1]
  df_trans = remove_collinear_features(df_train, coefficient_corr)
  print('Удалено признаков:')
  print(shape - df_trans.shape[1])

C4 | C1 | 0.97
C4 | C2 | 0.97
C6 | C1 | 0.98
C6 | C2 | 0.97
C6 | C4 | 0.96
C8 | C1 | 0.97
C8 | C2 | 0.98
C8 | C4 | 0.96
C10 | C1 | 0.96
C10 | C2 | 0.97
C10 | C4 | 0.95
C10 | C7 | 0.99
C10 | C8 | 1.0
C11 | C1 | 1.0
C11 | C2 | 0.99
C11 | C4 | 0.97
C11 | C6 | 0.99
C11 | C8 | 0.96
C12 | C7 | 1.0
C12 | C8 | 0.98
C12 | C10 | 0.98
C14 | C1 | 0.95
C14 | C6 | 0.98
C14 | C11 | 0.96
D6 | D4 | 0.96
D7 | D5 | 0.99
D12 | D4 | 1.0
D12 | D6 | 0.98
V21 | V17 | 0.95
V33 | V15 | 0.96
V50 | V31 | 0.98
V50 | V32 | 0.96
V57 | V15 | 0.96
V63 | V42 | 0.95
V71 | V31 | 0.96
V71 | V50 | 0.96
V84 | V21 | 0.96
V84 | V42 | 0.96
V84 | V63 | 0.96
V89 | V28 | 0.97
V92 | V31 | 0.97
V92 | V50 | 0.97
V92 | V71 | 0.96
V94 | V15 | 0.95
V94 | V33 | 0.98
V94 | V51 | 0.95
V101 | V95 | 1.0
V102 | V96 | 1.0
V102 | V97 | 0.99
V103 | V96 | 0.99
V103 | V97 | 1.0
V126 | V95 | 0.98
V126 | V101 | 0.98
V127 | V96 | 0.99
V127 | V97 | 0.98
V127 | V102 | 0.99
V127 | V103 | 0.98
V128 | V96 | 0.98
V128 | V97 | 0.99
V128 | V102 | 0.98
V128 

Разделение столбцов по типам и по заполненностью данными:

In [9]:
print(df_train.dtypes)

TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
                   ...   
id_34              object
id_35              object
id_36              object
id_37              object
id_38              object
Length: 429, dtype: object


In [10]:
from numpy.core.numerictypes import obj2sctype
def create_column_lists(data, max_nan=0.8):
  column_dict={}
  column_dict['categorical_column'] = []
  column_dict['numeric_column'] = []
  column_dict['float_column'] = []
  column_dict['bad_categorical_column'] = []
  column_dict['bad_numeric_column'] = []
  column_dict['bad_float_column'] = []
  for column in data.columns:
    row_count = data[column].shape[0]
    if data[column].isna().sum()/row_count > max_nan:
      if data[column].dtypes == object:
        column_dict['bad_categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['bad_float_column'].append(column)
        else:
          column_dict['bad_numeric_column'].append(column)
    else:
      if data[column].dtypes == object:
        column_dict['categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['float_column'].append(column)
        else:
          column_dict['numeric_column'].append(column)
  return column_dict




In [11]:
column_dict = create_column_lists(df_train, nan_percent)
for key in column_dict:
  print(key+':')
  print(column_dict[key])

categorical_column:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
numeric_column:
['card1']
float_column:
['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64

In [24]:
for key in column_dict:
  print(key+':')
  print(column_dict[key])

categorical_column:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'id_23', 'id_27']
numeric_column:
['card1']
float_column:
['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 

Разбираемся с почти пустыми столбцами

In [13]:
def fill_data(data,column_dict,is_drop_nan = False, is_test = False):
  if is_drop_nan == True:
    data.drop(columns=column_dict['bad_float_column']+
            column_dict['bad_numeric_column']+
            column_dict['bad_categorical_column'],axis = 1,inplace = True)  
  else:
    for column in column_dict['bad_categorical_column']:
      data[column].fillna(bad_fill_category_value, inplace = True)
    for column in column_dict['bad_float_column']:
      data[column].fillna(bad_fill_float_value, inplace = True)
    for column in column_dict['bad_numeric_column']:
      data[column].fillna(bad_fill_numeric_value, inplace = True)
    if is_test == False:
      column_dict['categorical_column'] = column_dict['categorical_column']+\
        column_dict['bad_categorical_column']
      column_dict['float_column'] = column_dict['float_column']+\
        column_dict['bad_float_column']
      column_dict['numeric_column'] = column_dict['numeric_column']+\
        column_dict['bad_numeric_column']

  """column_dict['bad_categorical_column']=[]
  column_dict['bad_float_column']=[]
  column_dict['bad_numeric_column']=[]"""

  for column in column_dict['categorical_column']:
    data[column].fillna(bad_fill_category_value, inplace = True)
  for column in column_dict['float_column']:
    data[column].fillna(data[column].mean(), inplace = True)
  for column in column_dict['numeric_column']:
    data[column].fillna(data[column].mean(), inplace = True)

  return data, column_dict


In [15]:
df_train,column_dict = fill_data(df_train,column_dict,is_drop_nan_columns, False)

Готовим данные

In [16]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size = 0.3, random_state = 42)


In [17]:
del df_train,df_y
gc.collect

<function gc.collect>

In [18]:
from sklearn.decomposition import PCA
if is_need_pca == True:
  pca = PCA(15)
  pca.fit(X_train[column_dict['float_column']+column_dict['numeric_column']])
  plt.plot(pca.explained_variance_ratio_.cumsum())
  plt.axvline(x=4)
  plt.grid()
  X_train_pca_part = pca.transform(X_train[column_dict['float_column']+column_dict['numeric_column']])
  X_val_pca_part = pca.transform(X_val[column_dict['float_column']+column_dict['numeric_column']])

CatBoost

In [19]:
!pip install -q catboost shap

In [20]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

model = CatBoostClassifier()
if is_need_pca == True: 
  model.fit(X_train_pca_part, y_train) 
  y_pred = model.predict(X_val_pca_part) 
else:
  model.fit(X_train[column_dict['float_column']+column_dict['numeric_column']], y_train) 
  y_pred = model.predict(X_val[column_dict['float_column']+column_dict['numeric_column']]) 
print(f"{roc_auc_score(y_pred, y_val)}")
 

Learning rate set to 0.134932
0:	learn: 0.4943229	total: 1.05s	remaining: 17m 31s
1:	learn: 0.3546477	total: 2.13s	remaining: 17m 44s
2:	learn: 0.2747348	total: 2.91s	remaining: 16m 7s
3:	learn: 0.2221633	total: 3.96s	remaining: 16m 27s
4:	learn: 0.1891102	total: 4.86s	remaining: 16m 7s
5:	learn: 0.1671098	total: 5.72s	remaining: 15m 47s
6:	learn: 0.1502516	total: 6.68s	remaining: 15m 47s
7:	learn: 0.1390413	total: 7.48s	remaining: 15m 27s
8:	learn: 0.1301802	total: 8.14s	remaining: 14m 56s
9:	learn: 0.1243470	total: 9.16s	remaining: 15m 6s
10:	learn: 0.1200537	total: 10.3s	remaining: 15m 27s
11:	learn: 0.1162353	total: 11.5s	remaining: 15m 46s
12:	learn: 0.1141336	total: 12.3s	remaining: 15m 35s
13:	learn: 0.1120221	total: 13.5s	remaining: 15m 51s
14:	learn: 0.1101630	total: 14.5s	remaining: 15m 50s
15:	learn: 0.1088035	total: 15.4s	remaining: 15m 49s
16:	learn: 0.1078082	total: 16.4s	remaining: 15m 48s
17:	learn: 0.1065366	total: 17.7s	remaining: 16m 4s
18:	learn: 0.1058272	total: 18

In [21]:
print(f"{roc_auc_score(y_pred, y_val)}")

0.9573968393984447


Тестовые данные

In [22]:
df_test, df_y1 = get_data('gdrive/My Drive/data/gp2/test_transaction.csv', 
                'gdrive/My Drive/data/gp2/test_identity.csv',
                True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [37]:
dict_col = {}
for column in df_test.columns:
  if column.startswith('id-'):
    dict_col[column] = column.replace("-","_")
df_test.rename(columns = dict_col,inplace=True)   

In [39]:
df_test = df_test[column_dict['float_column']+column_dict['numeric_column']+\
                  column_dict['categorical_column']]

In [40]:
df_test,column_dict = fill_data(df_test,column_dict,is_drop_nan_columns, True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [41]:
if is_need_pca == True:
  X_test_pca_part = pca.transform(df_test[column_dict['float_column']+column_dict['numeric_column']])
  Y_test_pred = model.predict(X_test_pca_part)
else:
  Y_test_pred = model.predict(df_test[column_dict['float_column']+column_dict['numeric_column']])
submission = pd.DataFrame(columns=['TransactionID'], data=df_y1 )
submission[target] = Y_test_pred
submission.head()
submission.isFraud.value_counts()

0    499502
1      7189
Name: isFraud, dtype: int64

In [42]:
submission.to_csv("submission.csv", index = False)

In [ ]:
"""train_pool = catboost.Pool(
    X_train,
    label=y_train,
    cat_features=f
)
val_pool = catboost.Pool(
    X_validate,
    label=y_validate,
    cat_features=f
)
test_pool = catboost.Pool(
    X_test,
    cat_features=f
)"""